# Image Preprocessing Pipeline – CIFAR-10

This notebook demonstrates a custom image preprocessing pipeline for color images.
Each transformation is applied step-by-step, visualized, and analyzed by printing
image shapes and pixel value ranges.


!pip install torch torchvision opencv-python matplotlib numpy

import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import cv2

dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

img1, _ = dataset[0]
img2, _ = dataset[1]


def show_image(img, title):
    img_np = img.permute(1, 2, 0).numpy()
    print(f"{title} → Shape: {img.shape}, Pixel Range: [{img.min():.3f}, {img.max():.3f}]")
    plt.imshow(img_np, cmap='gray')
    plt.title(title)
    plt.axis('off')


resize = transforms.Resize((64, 64))
img1_r = resize(img1)
img2_r = resize(img2)


gray = transforms.Grayscale(num_output_channels=1)
img1_g = gray(img1_r)
img2_g = gray(img2_r)

rotate = transforms.RandomRotation(30)
img1_rot = rotate(img1_g)
img2_rot = rotate(img2_g)

flip = transforms.RandomHorizontalFlip(p=1.0)
img1_f = flip(img1_rot)
img2_f = flip(img2_rot)


normalize = transforms.Normalize(mean=[0.5], std=[0.5])
img1_n = normalize(img1_f)
img2_n = normalize(img2_f)


def sharpen(img):
    img_np = img.squeeze().numpy()
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    sharp = cv2.filter2D(img_np, -1, kernel)
    return torch.tensor(sharp).unsqueeze(0)

img1_s = sharpen(img1_n)
img2_s = sharpen(img2_n)


steps = [
    ("Resized", img1_r, img2_r),
    ("Grayscale", img1_g, img2_g),
    ("Rotated", img1_rot, img2_rot),
    ("Flipped", img1_f, img2_f),
    ("Normalized", img1_n, img2_n),
    ("Sharpened", img1_s, img2_s)
]

plt.figure(figsize=(12, 10))
for i, (title, a, b) in enumerate(steps):
    plt.subplot(len(steps), 2, i*2 + 1)
    show_image(a, f"{title} - Image 1")
    plt.subplot(len(steps), 2, i*2 + 2)
    show_image(b, f"{title} - Image 2")

plt.tight_layout()
plt.show()


final_tensor = torch.stack([img1_s, img2_s])
print("Final Tensor Shape:", final_tensor.shape)